In [ ]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from IPython.core.pylabtools import figsize

sys.path.append('..//')
from utils_mitgcm import open_mitgcm_ds_from_config
from utils_energy_analysis import *

In [ ]:
model = 'dummy_rectangle'
mitgcm_config, ds = open_mitgcm_ds_from_config('..//config.json', model)

In [ ]:
folder_path = os.path.dirname(mitgcm_config['datapath'])
output_folder = os.path.join(folder_path, "energy_budget")
os.makedirs(output_folder, exist_ok=True)

In [ ]:
grid_resolution = 200
ds['YC'] = np.arange(1, len(ds['YC']) + 1) * grid_resolution - grid_resolution / 2
ds['XC'] = np.arange(1, len(ds['XC']) + 1) * grid_resolution - grid_resolution / 2
ds['YG'] = np.arange(0, len(ds['YG'])) * grid_resolution
ds['XG'] = np.arange(0, len(ds['XG'])) * grid_resolution

In [ ]:
%matplotlib inline
t=-1
plt.close('all')
plt.figure(figsize=(15,7))
ds_sel = ds['THETA'].isel(Z=0, time=t)
ds_sel.where(ds_sel != 0, np.nan).plot()
plt.title(ds_sel.time.values)
#plt.savefig(r"/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/figures/transect_location.png")
plt.show()

In [ ]:
aligned_u = ds.UVEL.rename({'XG':'XC'})
aligned_u['XC'] = ds['XC']

aligned_v = ds.VVEL.rename({'YG':'YC'})
aligned_v['YC'] = ds['YC']

aligned_w = ds.WVEL.rename({'Zl':'Z'})
aligned_w['Z'] = ds['Z']

# KE whole lake

In [ ]:
ke_tot = compute_ke(
    aligned_u,
    aligned_v,
    aligned_w,
    200,
    200,
    ds.drF)

In [ ]:
ke_tot.isel(time=10).sel(Z=-10, method='nearest').plot()

In [ ]:
ke_tot.sel(Z=-10, method='nearest').isel(YC=slice(0,80)).sum().values

In [ ]:
ke_tot.isel(time=10).isel(Z=range(60)).sum(dim=['XC','YC']).plot(y='Z', figsize=(4,10))
plt.title('Kinetic Energy')
#plt.ylabel('Kinetic energy [MJ]')

In [ ]:
df_ke_tot = ke_tot.sum(dim=['XC','YC','Z']).to_dataframe(name='ke_mj_total')['ke_mj_total']

In [ ]:
ke_tot.sum(dim=['XC','YC','Z']).plot()
plt.title('Kinetic Energy')
plt.ylabel('Kinetic energy [MJ]')

In [ ]:
df_ke_tot.iloc[10]

In [ ]:
df_ke_tot.iloc[:24].plot(figsize=(20,5))

In [ ]:
df_ke_tot.reset_index().to_csv(os.path.join(output_folder, "ke_whole_lake.csv"))

# KE surface

In [ ]:
ke_surf = compute_ke(aligned_u.isel(Z=0), aligned_v.isel(Z=0), aligned_w.isel(Z=0), 200, 200, 0.5)

In [ ]:
ke_surf.isel(time=-1).plot()
plt.hlines(11000,0,70000,'red')

In [ ]:
ke_surf.isel(time=-1).sel(YC=11000, method='nearest').plot()

for i in range(ke_surf.sizes['time']):
    plt.close('all')
    plt.figure(figsize=(15,7))
    ke_surf.isel(time=i).plot(vmax=1)
    plt.title(ke_surf.time.isel(time=i).values)
    plt.savefig(rf"/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/dummy_extended/seiche_analysis/map_total_ke_surface/ke_surface_time{i}.png")

In [ ]:
ke_surf.sum(dim=['XC','YC']).plot()

In [ ]:
total_ke_surf = ke_surf.sum(dim=['XC','YC'])

# Spectral analysis of total KE

In [ ]:
from utils_signal_processing import *
import pandas as pd

In [ ]:
df_total_ke = pd.read_csv(os.path.join(output_folder, "ke_whole_lake.csv"))
df_total_ke['time'] = pd.to_datetime(df_total_ke['time'])

xr_total_ke = df_total_ke.set_index("time")["ke_mj_total"].to_xarray()

In [ ]:
output_ke_folder = r"/home/leroquan@eawag.wroot.emp-eaw.ch/work_space/geneva_dummy_7ms/seiche_analysis/ke_analysis"

In [ ]:
xr_total_ke.plot()

In [ ]:
ke_fft = xr_compute_meanfft(xr_total_ke, M=1)

In [ ]:
cutoff1_hr = 138
cutoff2_hr = 3

cutoff1 = 1/(cutoff1_hr * 3600)
cutoff2 = 1/(cutoff2_hr * 3600)

In [ ]:
fig,ax = plot_freq_spectrum(ke_fft, 'U', depth=0, m_segm=1, y_lim_min=1e-9, x_lim_min=0.01e-4, fontsize=10)
ax.axvline(x = cutoff1, linestyle="--", color="k",label="cutoff1")
ax.axvline(x = cutoff2, linestyle="--", color="k",label="cutoff2")
ax.legend()

In [ ]:
ke_filtered = filter_signal_xarray(xr_total_ke, btype='bandpass', time_dim='time', dt=3600, period_cutoff_low=(cutoff1_hr*3600), period_cutoff_high=(cutoff2_hr*3600), order=5)

In [ ]:
ke_filtered.plot()

In [ ]:
xr_total_ke.plot()
ke_filtered.plot()
(xr_total_ke - ke_filtered).plot()
plt.legend()